### Load data

In [1]:
import pandas as pd
import numpy as np

na_values = ['NO CLUE', 'N/A', '0']
# the request is not a dataframe but
# TextFileReader object for iteration
requests = pd.read_csv('311_data.csv',
                       chunksize=1000000,
                       na_values=na_values,
                       low_memory=False,
                       dtype={'Incident Zip': str})

def chunk_data(requests):
    chunk_list = []  # append each chunk df here
    # Each chunk is in df format
    for chunk in requests:
        chunk_list.append(chunk)
        # concat the list into dataframe
    df_concat = pd.concat(chunk_list)

    return df_concat

def fix_zip_codes(zips):
    # Truncate everything to length 5
    zips = zips.str.slice(0, 5)

    # Set 00000 zip codes to nan
    zero_zips = zips == '00000'
    zips[zero_zips] = np.nan

    return zips


df_req = chunk_data(requests)
df_req['Incident Zip'] = fix_zip_codes(df_req['Incident Zip'])


# Look at data after fixing zips

In [2]:
df_req.rename(columns={"Agency Name": "AgencyName", "Incident Zip": "IncidentZip", "Complaint Type": "ComplaintType"})
# Rename column names for easy access
df_req.columns = df_req.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

dfObj_agency = df_req.agency.unique() #Find unique elements in the column
dfObj_agency_name = df_req.agency_name.unique() #Find unique elements in the column
dfObj_incident_zip = df_req.incident_zip.unique() #Find unique elements in the column
dfObj_complaint_type = df_req.complaint_type.unique() #Find unique elements in the column

# print unique values in these columns:
print('agency ' + str(len(dfObj_agency)))
print('agency_name ' + str(len(dfObj_agency_name)))
print('incident_zip ' + str(len(dfObj_incident_zip)))
print('complaint_type ' + str(len(dfObj_complaint_type)))


print(dfObj_incident_zip) #Review unique names for missing data or odd format
print(dfObj_agency) #Review unique names for missing data or odd format

agency 31
agency_name 1933
incident_zip 2392
complaint_type 434
[nan '11216' '10012' ... '11519' '01650' '45201']
['DOF' 'NYPD' 'DOHMH' 'DPR' 'HRA' 'TLC' 'DOT' 'DFTA' 'DSNY' 'HPD' 'DOB'
 'DEP' 'DHS' 'DCA' 'NYCEM' 'DOE' '3-1-1' 'EDC' 'DOITT' 'DCAS' 'ACS' 'TAX'
 'FDNY' 'DCP' 'DORIS' 'COIB' 'TAT' 'CEO' 'MOC' 'OMB' 'DVS']


# Perform more preprocessing remove some odd format

In [3]:
df_req.incident_zip.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
df_req.complaint_type.replace(regex=True,inplace=True,to_replace=r'[^a-zA-Z ]+',value=r'')

In [ ]:
df_req.to_csv('nyc311_filter.csv', index=False)

In [5]:
# Look at data format
import sys
np.set_printoptions(threshold=sys.maxsize)
print(df_req.complaint_type.unique())
print(df_req.incident_zip.unique())

['SCRIE' 'Illegal Parking' 'DOF Property  Payment Issue'
 'Indoor Air Quality' 'Dead Tree' 'Benefit Card Replacement'
 'For Hire Vehicle Complaint' 'Sidewalk Condition'
 'Housing  Low Income Senior' 'Traffic' 'Street Sign  Missing'
 'DOF Property  Request Copy' 'Graffiti' 'APPLIANCE' 'DOORWINDOW'
 'FLOORINGSTAIRS' 'GENERAL' 'Elder Abuse'
 'Case Management Agency Complaint' 'Street Condition'
 'Street Sign  Dangling' 'Food Establishment' 'DOF Property  Owner Issue'
 'DOF Parking  Request Copy' 'Noise  Vehicle' 'Cranes and Derricks'
 'Electrical' 'Scaffold Safety' 'DOF Parking  Payment Issue'
 'Request Large Bulky Item Collection' 'Water System' 'HEATHOT WATER'
 'HPD Literature Request' 'RootSewerSidewalk Condition'
 'Illegal Tree Damage' 'Overgrown TreeBranches' 'Asbestos'
 'DHS Advantage  Tenant' 'DHS Advantage LandlordBroker' 'Smoking'
 'Blocked Driveway' 'Consumer Complaint' 'Tattooing' 'Taxi Complaint'
 'Noise  Residential' 'Noise  StreetSidewalk' 'Maintenance or Facility'
 'DCA  DO

[nan '11216' '10012' '10456' '10023' '10022' '10036' '10032' '10028'
 '10304' '10009' '11360' '11205' '11221' '10002' '10010' '11232' '11234'
 '11362' '10471' '11361' '11363' '10463' '11423' '11367' '10303' '11354'
 '10308' '11226' '10475' '11224' '11365' '11432' '11233' '10474' '11435'
 '10454' '10309' '11219' '11102' '10302' '11364' '10461' '10306' '11209'
 '11433' '10021' '10075' '10128' '11213' '10014' '11356' '10034' '10027'
 '10031' '11217' '10026' '10452' '11415' '10038' '11434' '11215' '11201'
 '11427' '10019' '11375' '11218' '11691' '11414' '10307' '10065' '11355'
 '10466' '10024' '11374' '11220' '11204' '10457' '11103' '10469' '11377'
 '11412' '11207' '11237' '10017' '10025' '10472' '10462' '10030' '11203'
 '10458' '11235' '11385' '11212' '11225' '11229' '10011' '10301' '10040'
 '11228' '10033' '10162' '11358' '11236' '11428' '11370' '10467' '10016'
 '11222' '11210' '10305' '11366' '11230' '11373' '11368' '11378' '11101'
 '11223' '11208' '10035' '11422' '10029' '11238' '10468

# What are the number of complaints per department for each zip code?

In [8]:
# total complaints count
df_count = df_req.groupby(['complaint_type','incident_zip', 'agency']).size(
).reset_index(name="number_of_complaints").sort_values(by='number_of_complaints', ascending=0)
pd.set_option('display.max_rows', 50000)  # or 1000
df_count.head(50)

,complaint_type,incident_zip,agency,number_of_complaints
26005,Noise Residential,10467,NYPD,41504
26053,Noise Residential,11226,NYPD,40430
18115,HEATHOT WATER,11226,HPD,38392
26006,Noise Residential,10468,NYPD,36095
18062,HEATHOT WATER,10458,HPD,35826
18071,HEATHOT WATER,10467,HPD,34917
25996,Noise Residential,10458,NYPD,31790
18072,HEATHOT WATER,10468,HPD,31440
18057,HEATHOT WATER,10453,HPD,31389
25941,Noise Residential,10031,NYPD,31228
